In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os, sys

In [3]:
import pandas as pd

In [4]:
sys.path.append('..')

from pyWebOmics.constants import *
from pyWebOmics.mapping import Mapper
from pyWebOmics.common import set_log_level_info

2021-01-28 20:02:26.220 | INFO     | pyWebOmics.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# Demonstration of pyMultiOmics

## Load the processed Zebrafish data from [1]

[1] [Rabinowitz, Jeremy S., et al. "Transcriptomic, proteomic, and metabolomic landscape of positional memory in the caudal fin of zebrafish." Proceedings of the National Academy of Sciences 114.5 (2017): E717-E726.](https://www.pnas.org/content/114/5/E717.short)

In [5]:
DATA_FOLDER = os.path.abspath(os.path.join('test_data', 'zebrafish_data'))
DATA_FOLDER

'C:\\Users\\joewa\\Work\\git\\pyWebOmics\\notebooks\\test_data\\zebrafish_data'

Read transcriptomics data (identified by their gene ids)

In [6]:
gene_data = pd.read_csv(os.path.join(DATA_FOLDER, 'gene_data_combined.csv'), index_col='Identifier')
gene_design = pd.read_csv(os.path.join(DATA_FOLDER, 'gene_design.csv'), index_col='sample')

In [7]:
gene_data.head()

,US-1584693,US-1584700,US-1584706,US-1584712,US-1584722,US-1584724,US-1584725,US-1584732,US-1584738,US-1584744,...,US-1584753,US-1584754,US-1584758,US-1584765,FC_distal_vs_proximal,padj_distal_vs_proximal,FC_distal_vs_middle,padj_distal_vs_middle,FC_middle_vs_proximal,padj_middle_vs_proximal
Identifier,,,,,,,,,,,,,,,,,,,,,
ENSDARG00000000001,51,40,69,78,89,47,88,86,110,55,...,58,104,43,72,0.869331,0.000008,0.748943,4.380000e-05,0.114026,0.630834
ENSDARG00000000002,283,129,164,269,211,171,146,256,283,150,...,142,272,260,256,0.287823,0.031298,1.005337,1.310000e-13,-0.724987,0.000001
ENSDARG00000000018,545,503,547,387,332,559,623,499,436,488,...,462,287,495,299,-0.437271,0.000389,-0.404770,6.868400e-04,-0.040193,0.760679
ENSDARG00000000019,437,469,538,557,550,404,544,443,623,502,...,470,460,329,480,0.521291,0.000015,0.271082,1.936266e-02,0.242435,0.041606
ENSDARG00000000068,266,249,247,236,195,247,283,259,299,232,...,231,236,274,241,0.064820,0.595522,0.142243,2.579239e-01,-0.084764,0.528336


In [8]:
gene_design

,group
sample,
US-1584752,Proximal
US-1584732,Proximal
US-1584724,Proximal
US-1584693,Proximal
US-1584758,Proximal
US-1584725,Middle
US-1584706,Middle
US-1584700,Middle
US-1584744,Middle


Read proteomics data

In [9]:
protein_data = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_data.csv'), index_col='Uniprot')
protein_design = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_design.csv'), index_col='sample')

In [10]:
protein_data.head()

,Distal#3_01,Distal#3_02,Distal#3_03,Distal#3_04,Middle#3_01,Middle#3_02,Middle#3_03,Middle#3_04,Proximal#3_01,Proximal#3_02,Proximal#3_03,Proximal#3_04
Uniprot,,,,,,,,,,,,
A0A0A0MPL4,9.955264e+05,4.946580e+06,1.377194e+06,2.208140e+06,2.907807e+06,4.231976e+06,1.560849e+06,2.852904e+06,1781795.086,2.668135e+06,3.079148e+06,2.840473e+06
A0A0B4J1A5,2.982519e+06,8.816655e+06,7.668431e+06,4.632309e+06,7.672153e+06,7.776017e+06,6.633781e+06,8.242783e+06,5475654.544,5.703832e+06,8.294364e+06,1.334874e+07
A0A0B4J1A7,1.553049e+07,1.037155e+06,1.856137e+07,1.767859e+07,1.375736e+07,1.747998e+07,1.517507e+07,2.394465e+07,3157387.719,1.794775e+07,2.300430e+07,2.063800e+07
A0AUQ3,2.012699e+06,3.088982e+06,2.455865e+06,9.448331e+05,2.866780e+06,2.661669e+06,2.100352e+06,2.133662e+06,1738244.989,2.629396e+06,2.900560e+06,2.416018e+06
A0AUR9,3.640487e+06,2.588477e+07,3.415989e+07,2.868569e+06,1.971142e+06,2.472776e+06,5.615177e+06,1.303356e+06,3263299.566,6.866769e+06,2.465929e+06,4.515643e+06


In [11]:
protein_design

,group
sample,
Distal#3_01,Distal
Distal#3_02,Distal
Distal#3_03,Distal
Distal#3_04,Distal
Middle#3_01,Middle
Middle#3_02,Middle
Middle#3_03,Middle
Middle#3_04,Middle
Proximal#3_01,Proximal


Read metabolomics data

In [12]:
compound_data = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_data_kegg.csv'), index_col='Identifier')
compound_design = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_design.csv'), index_col='sample')

In [13]:
compound_data.head()

,distal_M1,distal_M2,distal_M3,distal_F1,distal_F2,distal_F3,middle_M1,middle_M2,middle_M3,middle_F1,middle_F2,middle_F3,proximal_M1,proximal_M2,proximal_M3,proximal_F1,proximal_F2,proximal_F3
Identifier,,,,,,,,,,,,,,,,,,
C00565,75170.0,57052,39170.0,84057,38608.0,64126.0,50214.0,75680,165178,121856,77061,98015.0,113765,96098,84198,117644,169459,169669
C00037,64511.0,33658,23565.0,52102,49508.0,37498.0,30417.0,55728,88519,103871,45974,73101.0,72725,66008,54220,95341,110192,291598
C01104,5787534.0,4351239,4401036.0,8187282,8431125.0,5082056.0,5138937.0,7341351,7837293,9256269,9934066,10243285.0,7344406,5524811,4809250,9279874,9047339,9211255
C00134,3430897.0,1877785,1225710.0,2326620,2421267.0,2595529.0,2003627.0,2120053,2269318,3220850,4596854,3155377.0,3760854,2658833,2488025,2506550,4000703,3292566
C00213,112845.0,129977,122292.0,63219,50113.0,100343.0,156651.0,176682,379322,160906,56802,107161.0,235982,181200,142994,116132,94589,167280


In [14]:
compound_design

,group
sample,
distal_M1,Distal
distal_M2,Distal
distal_M3,Distal
distal_F1,Distal
distal_F2,Distal
distal_F3,Distal
middle_M1,Middle
middle_M2,Middle
middle_M3,Middle


In [15]:
set_log_level_info()

1

## Create a mapping object

The mapping object uses Reactome to map the different biological entities in the data:
- Transcripts (or genes) are connected to the proteins they encode
- Proteins and compounds are connected to reactions they're involved in
- Reactions are connected to pathways

In [16]:
m = Mapper(DANIO_RERIO, metabolic_pathway_only=True) \
        .set_gene(gene_data, gene_design) \
        .set_protein(protein_data, protein_design) \
        .set_compound(compound_data, compound_design) \
        .build()

2021-01-28 20:02:27.425 | INFO     | pyWebOmics.functions:reactome_mapping:72 - Mapping genes -> proteins
2021-01-28 20:02:28.319 | INFO     | pyWebOmics.functions:reactome_mapping:77 - Mapping proteins -> reactions
2021-01-28 20:02:30.422 | INFO     | pyWebOmics.functions:reactome_mapping:85 - Mapping compounds -> reactions
2021-01-28 20:02:31.325 | INFO     | pyWebOmics.functions:reactome_mapping:91 - Mapping reactions -> pathways
2021-01-28 20:02:31.644 | INFO     | pyWebOmics.functions:reactome_mapping:102 - Mapping reactions -> proteins
2021-01-28 20:02:34.018 | INFO     | pyWebOmics.functions:reactome_mapping:109 - Mapping reactions -> compounds
2021-01-28 20:02:35.294 | INFO     | pyWebOmics.functions:reactome_mapping:121 - Mapping proteins -> genes
2021-01-28 20:02:49.355 | INFO     | pyWebOmics.mapping:_add_nodes:177 - Processing nodes: genes
2021-01-28 20:02:52.063 | INFO     | pyWebOmics.mapping:_add_nodes:177 - Processing nodes: proteins
2021-01-28 20:02:54.367 | INFO     |

In [17]:
m

## Querying mapping object

Below shows some example queries we can perform with the mapping object

##### Find out the number of entities mapped to each reaction

In [18]:
complete = False

reactions = m.get_nodes(types=REACTIONS, complete=complete)
count = 0
for reaction_id, reaction_data in reactions:
    reaction_name = reaction_data['display_name']
    genes = m.get_connected(reaction_id, GENOMICS, complete=complete)
    proteins = m.get_connected(reaction_id, PROTEOMICS, complete=complete)
    compounds = m.get_connected(reaction_id, METABOLOMICS, complete=complete)
    
    if len(genes) > 0 and len(proteins) > 0 and len(compounds) > 0:
        print('%s\t%s (genes=%d proteins=%d compounds=%d)' % (reaction_id, reaction_name, len(genes), len(proteins), len(compounds)))
        count += 1
        if count >= 10:
            break

R-DRE-109278	Nt5e:zn2+ hydrolyses amp,damp,gmp, imp (genes=1 proteins=1 compounds=6)
R-DRE-109291	Cmp or tmp or ump + h2o => cytidine, thymidine, or uridine + orthophosphate [nt5e] (genes=1 proteins=1 compounds=4)
R-DRE-109624	(2-deoxy)adenosine + atp => (d)amp + adp (adk) (genes=1 proteins=1 compounds=2)
R-DRE-110141	(d)adp + adp <=> (d)amp + atp (ak1) (genes=1 proteins=1 compounds=1)
R-DRE-110144	Adp + adp <=> amp + atp [ak2] (genes=1 proteins=1 compounds=1)
R-DRE-110145	Amp + atp <=> adp + adp [ak2] (genes=1 proteins=1 compounds=1)
R-DRE-111524	Imp + l-aspartate + gtp => adenylosuccinate + gdp + pi [adss] (genes=1 proteins=1 compounds=4)
R-DRE-1237160	Mta is cleaved and phosphorylated (genes=1 proteins=1 compounds=1)
R-DRE-1247910	Cndp2:2mn2+ dimer hydrolyses cysgly (genes=1 proteins=1 compounds=1)
R-DRE-1482976	Cdp-dag is converted to pi by cdipt (genes=1 proteins=1 compounds=2)


##### What are the nodes directly connected to reaction 'R-DRE-71735' in the graph?

In [19]:
query_id = 'R-DRE-71735'
m.get_neighbours(query_id)

['F1QAA7', '16027', 'R-DRE-71384']

##### Find genes connected to protein 'F1QAA7'

In [20]:
query_id = 'F1QAA7'
m.get_connected(query_id, GENOMICS)

['ENSDARG00000037781']

##### Find compounds connected to protein 'F1QAA7'

In [21]:
query_id = 'F1QAA7'
m.get_connected(query_id, METABOLOMICS)

['16027']

##### Find genes connected to compound '16027'

In [22]:
query_id = '16027'
m.get_connected(query_id, GENOMICS)

['ENSDARG00000014674',
 'ENSDARG00000100731',
 'ENSDARG00000078073',
 'ENSDARG00000053481',
 'ENSDARG00000006546',
 'ENSDARG00000079848',
 'ENSDARG00000019924',
 'ENSDARG00000015524',
 'ENSDARG00000071017',
 'ENSDARG00000087927',
 'ENSDARG00000039429',
 'ENSDARG00000061889',
 'ENSDARG00000005926',
 'ENSDARG00000063525',
 'ENSDARG00000009250',
 'ENSDARG00000101631',
 'ENSDARG00000037781',
 'ENSDARG00000016375',
 'ENSDARG00000044142']

##### Find pathways connected to genes 'ENSDARG00000087927'

In [23]:
query_id = 'ENSDARG00000087927'
m.get_connected(query_id, PATHWAYS)

['R-DRE-2393930']